<a href="https://colab.research.google.com/github/rikunemu/data_study/blob/main/pose_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  mediapipeで取った特徴点を分析

90分対策  
F12を開いて以下のスクリプトをコンソールで実行  
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,600000)

  
10分ごとに接続

参考url:
https://sunnyday-travel-aso-6487.ssl-lolipop.jp/programing/python/keras/lstm-use/

In [1]:
import sys
from google.colab import drive
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import csv
import os
import keras
from keras import models
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.recurrent import LSTM
#from keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils as np_utils
from keras.utils.np_utils import to_categorical
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
#from tensorflow.keras.initializers import glorot_uniform
#from tensorflow.keras.initializers import orthogonal
#from tensorflow.keras.initializers import TruncatedNormal
drive.mount("/content/drive")

Mounted at /content/drive


## csvデータ読み込み
train,test,validそれぞれ7感情の特徴点を記録

In [2]:
fp_train='/content/drive/MyDrive/data分析/join_csv/csvtrainresult.csv'
df_train=pd.read_csv(fp_train)
fp_test='/content/drive/MyDrive/data分析/join_csv/csvtestresult.csv'
df_test=pd.read_csv(fp_test)

Unnameはいらないので削除

In [3]:
df_test=df_test.replace('NAN',0.1)
df_train=df_train.replace('NAN',0.1)

In [4]:
df_train=df_train.drop(columns=df_train.columns[[0]],axis=1)

967,5,17

In [5]:
df_test=df_test.drop(columns=df_test.columns[[0]],axis=1)

3659,4  
30,487

In [6]:
df_train=df_train.fillna(0.1)
df_test=df_test.fillna(0.1)

それぞれのデータを目的変数と説明変数に分ける

In [7]:
x_train=DataFrame(df_train.drop("correct",axis=1)).values
Y_train=DataFrame(df_train["correct"]-1).values
x_test=DataFrame(df_test.drop("correct",axis=1)).values
Y_test=DataFrame(df_test["correct"]-1).values

In [8]:
def build_model(maxlen):
  model = Sequential()
  model.add(LSTM(64, return_sequences=False, activation='tanh', input_shape=(30,maxlen)))
  model.add(Dropout(0.1))
  #model.add(LSTM(128, return_sequences=True, activation='tanh'))
  #model.add(Dropout(0.1))
  #model.add(LSTM(64, return_sequences=False, activation='tanh'))
  #model.add(Dropout(0.1))
  #model.add(Dense(64, activation='relu'))

  #model.add(Dense(32, activation='relu'))

  model.add(Dense(20, activation='softmax'))
  adam = Adam(learning_rate=1e-4)
  model.compile(optimizer=adam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [10]:
FOLD = 10
EPOCH = 100
BATCH_SIZE = 10

valid_scores = []
models = []
kf = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=42)

for fold, (train_indices, valid_indices) in enumerate(kf.split(x_train, Y_train)):
    X_train, X_valid = x_train[train_indices], x_train[valid_indices]
    y_train, y_valid = Y_train[train_indices], Y_train[valid_indices]
    

    X_train = np.array(X_train).reshape(720, 30, 162)
    X_valid = np.array(X_valid).reshape(80, 30,162)
    y_train = np.array(y_train[::30]).reshape(720)
    y_valid = np.array(y_valid[::30]).reshape(80)
    y_train = np_utils.to_categorical(y_train,20)
    y_valid = np_utils.to_categorical(y_valid,20)

    X_train = X_train.astype(np.float)
    X_valid = X_valid.astype(np.float)
    y_train = y_train.astype(np.float)
    y_valid = y_valid.astype(np.float)
    model = build_model(X_train.shape[2])

    #model = build_model(X_train.shape[1])
    rlr = ReduceLROnPlateau(monitor='val_loss',
                            factor=0.1,
                            patience=3,
                            verbose=0,
                            min_delta=1e-4,
                            mode='max')
    ckp = ModelCheckpoint(f'model_{fold}.hdf5',
                          monitor='val_loss',
                          verbose=0,
                          save_best_only=True,
                          save_weights_only=True,
                          mode='max')
    es = EarlyStopping(monitor='val_loss',
                       min_delta=1e-4,
                       patience=7,
                       mode='max',
                       baseline=None,
                       restore_best_weights=True,
                       verbose=0)

    history=model.fit(X_train, y_train,
              validation_data=(X_valid, y_valid),
              epochs=EPOCH,
              batch_size=BATCH_SIZE)
              callbacks=[rlr, ckp, es])
              verbose=0)

    y_valid_pred = model.predict(X_valid)
    score = log_loss(y_valid, y_valid_pred)
    print(f'fold {fold} log loss: {score}')
    valid_scores.append(score)

    models.append(model)

cv_score = np.mean(valid_scores)
print(f'CV score: {cv_score}')


Epoch 1/100
72/72 [==============================] - 3s 22ms/step - loss: 3.0108 - accuracy: 0.0611 - val_loss: 2.9534 - val_accuracy: 0.0875
Epoch 2/100
72/72 [==============================] - 1s 17ms/step - loss: 2.9363 - accuracy: 0.1111 - val_loss: 2.8868 - val_accuracy: 0.0875
Epoch 3/100
72/72 [==============================] - 1s 17ms/step - loss: 2.8421 - accuracy: 0.1528 - val_loss: 2.7373 - val_accuracy: 0.2500
Epoch 4/100
72/72 [==============================] - 1s 17ms/step - loss: 2.7233 - accuracy: 0.1903 - val_loss: 2.6068 - val_accuracy: 0.3250
Epoch 5/100
72/72 [==============================] - 1s 18ms/step - loss: 2.5545 - accuracy: 0.2472 - val_loss: 2.3620 - val_accuracy: 0.3875
Epoch 6/100
72/72 [==============================] - 1s 18ms/step - loss: 2.3788 - accuracy: 0.3000 - val_loss: 2.1565 - val_accuracy: 0.4250
Epoch 7/100
72/72 [==============================] - 1s 17ms/step - loss: 2.2452 - accuracy: 0.3389 - val_loss: 2.0182 - val_accuracy: 0.4375
Epoch 